# Use this as a reference to pull data from LIMS

In [1]:
import pg8000          #pg8000 access SQL databases
import pandas as pd    #pandas will be needed to work in a dataframe

In [2]:
#code from Agata
#these are nice functions to open LIMS, make a query and then close LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results


#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

In [19]:
my_query = "SELECT * FROM specimens LIMIT 10"
#we are going to select all columns in the specimens table and limit our search to the first 10

my_result = limsquery(my_query)

first_element = my_result[0]
#now we only want to look at the first element of our result

print first_element

{'cell_depth': None, 'ephys_roi_result_id': None, 'parent_y_coord': 0, 'reference_space_id': None, 'updated_at': datetime.datetime(2016, 12, 16, 4, 54, 44, 477335), 'cell_label': None, 'preparation_method_id': None, 'parent_x_coord': 2, 'location_id': None, 'id': 556516441, 'cortex_layer_id': None, 'plane_of_section_id': 11, 'frozen_at': None, 'flipped_specimen_id': 561557765, 'data': None, 'pinned_radius': None, 'rna_integrity_number': None, 'histology_well_name': None, 'created_by': None, 'priority': None, 'parent_id': 556516212, 'ephys_start_time_sec': None, 'project_id': 305094322, 'alignment3d_id': None, 'carousel_well_name': u'T301_122_161107_01_12', 'patched_cell_container': None, 'updated_by': None, 'cell_prep_id': None, 'biophysical_model_state': u'review_required', 'barcode': u'0556516441', 'storage_directory': None, 'x_coord': None, 'tissue_ph': None, 'specimen_preparation_method_id': None, 'donor_id': 555257198, 'operation_id': None, 'ephys_neural_tissue_plan_id': 555257244

In [29]:
my_query = "SELECT * FROM donors LIMIT 10"
#we are going to select all columns in the specimens table and limit our search to the first 10

my_result = limsquery(my_query)

first_element = my_result[0]
#now we only want to look at the first element of our result

print first_element.keys()

['weight', 'education_level_id', 'updated_at', 'height', 'transgenic_induction_method_id', 'id', 'death_on', 'primary_tissue_source_id', 'full_genotype', 'induction_method', 'age_id', 'created_by', 'occupation_id', 'date_of_birth', 'external_donor_name', 'race_id', 'death_manner_id', 'updated_by', 'organism_id', 'data', 'name', 'created_at', 'gender_id', 'baseline_weight_g', 'death_cause_id', 'handedness_id']


In [21]:
my_query = "SELECT * FROM well_known_files LIMIT 10"
my_result = limsquery(my_query)
first_element = my_result[0]
print first_element.keys()

['published_at', 'workflow_state', 'created_at', 'updated_at', 'filename', 'storage_directory', 'file_source_id', 'attachable_type', 'content_type', 'well_known_file_type_id', 'attachable_id', 'id', 'size']


In [22]:
my_query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES"
my_result = limsquery(my_query)
my_result

[{'table_name': u'pg_type'},
 {'table_name': u'pg_roles'},
 {'table_name': u'pg_group'},
 {'table_name': u'pg_user'},
 {'table_name': u'pg_rules'},
 {'table_name': u'pg_views'},
 {'table_name': u'pg_tables'},
 {'table_name': u'pg_matviews'},
 {'table_name': u'pg_indexes'},
 {'table_name': u'pg_stats'},
 {'table_name': u'pg_settings'},
 {'table_name': u'pg_locks'},
 {'table_name': u'pg_cursors'},
 {'table_name': u'pg_available_extensions'},
 {'table_name': u'pg_available_extension_versions'},
 {'table_name': u'pg_prepared_xacts'},
 {'table_name': u'pg_prepared_statements'},
 {'table_name': u'pg_seclabels'},
 {'table_name': u'pg_timezone_abbrevs'},
 {'table_name': u'pg_timezone_names'},
 {'table_name': u'pg_stat_all_tables'},
 {'table_name': u'pg_stat_xact_all_tables'},
 {'table_name': u'pg_stat_sys_tables'},
 {'table_name': u'pg_stat_xact_sys_tables'},
 {'table_name': u'pg_stat_user_tables'},
 {'table_name': u'pg_stat_xact_user_tables'},
 {'table_name': u'pg_statio_all_tables'},
 {'tabl

In [7]:
#This is the basics of how a SQL query looks
#SELECT TableA., TableB., TableC., TableD. FROM TableA JOIN TableB ON TableB.aID = TableA.aID 
#JOIN TableC ON TableC.cID = TableB.cID WHERE DATE(TableC.date)=date(now())
#This allows us to pull in info from 4 different tables since everything we want is not 
#stored in one place, but there are overlaps to link things up. 10 digit IDs generally are your link

In [8]:
# note both specimens and ephys roi results have an ID column, so we renamed the specimens column 
# as cell_id in the output

lims_query = "SELECT ephys_roi_results.id, specimens.id AS cell_id, specimens.name, specimens.ephys_roi_result_id \
FROM ephys_roi_results JOIN specimens ON specimens.ephys_roi_result_id = ephys_roi_results.id"
lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

ephys_roi_result_id    cell_id         id  \
13904            668475134  668475152  668475134   
13905            668475449  668475465  668475449   
13906            668489557  668489573  668489557   
13907            668531759  668531764  668531759   
13908            668536310  668536315  668536310   

                                                    name  
13904              Slc17a6-IRES-Cre;Ai14-376996.04.01.01  
13905              Slc17a6-IRES-Cre;Ai14-376996.04.02.02  
13906              Ndnf-IRES2-dgCre;Ai14-378475.04.02.01  
13907  Ntsr1-Cre_GN220;Ai140;Sst-IRES-FlpO-377972-Ai6...  
13908  Ntsr1-Cre_GN220;Ai140;Sst-IRES-FlpO-377972-Ai6...

In [9]:
#This is the structure we want to follow for SQL:
#SELECT your tables.columns
#FROM these columns are coming if you are going to starting using
#JOIN left ON right
#WHERE gives you the ability to filter data
#AND and OR add variability to your filters
#" opens and closes your query"
#\ lets you wrap text
#We can shorten ephys_roi_results to err and specimens to s, search for all of our features and use
#JOIN to join the search results from ephys_roi_results and specimens following the outline from above

In [10]:
# callout the abreveations in the FROM section
# ephys_roi_results err

lims_query = "SELECT err.id, s.id AS cell_id, s.name \
FROM ephys_roi_results err \
JOIN specimens s ON s.ephys_roi_result_id = err.id"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

cell_id         id                                               name
13904  668475152  668475134              Slc17a6-IRES-Cre;Ai14-376996.04.01.01
13905  668475465  668475449              Slc17a6-IRES-Cre;Ai14-376996.04.02.02
13906  668489573  668489557              Ndnf-IRES2-dgCre;Ai14-378475.04.02.01
13907  668531764  668531759  Ntsr1-Cre_GN220;Ai140;Sst-IRES-FlpO-377972-Ai6...
13908  668536315  668536310  Ntsr1-Cre_GN220;Ai140;Sst-IRES-FlpO-377972-Ai6...

In [11]:
# callout the abreveations in the FROM section
# ephys_roi_results err

lims_query = "SELECT err.id, s.id, s.name, proj.code \
FROM ephys_roi_results err \
JOIN specimens s ON s.ephys_roi_result_id = err.id \
JOIN projects proj ON s.project_id = proj.id \
WHERE proj.code = 'H301'"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

code         id                 name
908  H301  643584484  H17.06.013.12.04.01
909  H301  643589588  H17.06.013.12.04.02
910  H301  643601251  H17.06.013.12.13.01
911  H301  643619982  H17.06.013.12.13.02
912  H301  643625553  H17.06.013.12.13.03

In [12]:
#filters for cells that only failed due to access resistance over 20 MOhm
#grabs the ephys roi result id, project code, recording date, cell name, cell id, and qc results

lims_query = "SELECT err.id, err.recording_date, err.failed_bad_rs, \
err.failed_electrode_0, err.failed_clogged_pipette, err.failed_no_seal, \
err.failed_other, err.workflow_state, s.name, s.ephys_roi_result_id, s.project_id, proj.id, proj.code \
FROM ephys_roi_results err JOIN specimens s ON s.ephys_roi_result_id = err.id \
JOIN projects proj ON s.project_id = proj.id \
WHERE (failed_bad_rs = 'TRUE' \
AND err.failed_electrode_0 = 'FALSE' \
AND err.failed_clogged_pipette = 'FALSE' \
AND err.failed_no_seal = 'FALSE' \
AND err.failed_other = 'FALSE' \
AND err.workflow_state != 'manual_passed') \
AND (proj.code = 'T301' OR proj.code = 'T301x')"

lims_df = get_lims_dataframe(lims_query)
lims_df.tail()

ephys_roi_result_id   code      recording_date  failed_bad_rs  \
67            601831626   T301 2017-07-10 21:03:11           True   
68            602660919   T301 2017-07-13 22:16:27           True   
69            604693290  T301x 2017-07-26 22:50:46           True   
70            605537141  T301x 2017-07-28 20:37:50           True   
71            605538385  T301x 2017-07-28 21:32:11           True   

   workflow_state                                 name  \
67  manual_failed    Nos1-CreERT2;Ai14-333287.03.01.01   
68  manual_failed  Scnn1a-Tg2-Cre;Ai14-333165.04.02.01   
69  manual_failed  Rbp4-Cre_KL100;Ai14-337193.04.02.01   
70  manual_failed   Gad2-IRES-Cre;Ai14-336420.04.02.01   
71  manual_failed   Gad2-IRES-Cre;Ai14-336420.02.02.01   

    failed_clogged_pipette  failed_other  project_id  failed_electrode_0  \
67                   False         False   305094322               False   
68                   False         False   305094322               False   
69                   False         False   300080300               False   
70                   False         False   300080300               False   
71                   False         False   300080300               False   

           id  failed_no_seal  
67  305094322           False  
68  305094322           False  
69  300080300           False  
70  300080300           False  
71  300080300           False

In [15]:
query = "SELECT * \
FROM scheduled_procedures \
JOIN users ON users.id = scheduled_procedures.assigned_to_id \
JOIN scheduled_procedures_subjects sps ON scheduled_procedures.id = sps.scheduled_procedure_id \
JOIN specimens ON sps.subject_id = specimens.id \
WHERE users.login = 'nadiad' and scheduled_procedures.lab_request_status ='active' \
ORDER BY scheduled_procedures.created_at"
df = get_lims_dataframe(query)
df.head()

cell_depth ephys_roi_result_id siv_default_ontology_id scheduled_date  \
0       None                None                    None           None   
1       None                None                    None           None   
2       None                None                    None           None   
3       None                None                    None           None   
4       None                None                    None           None   

  parent_y_coord  reference_space_id task_flow_id                 updated_at  \
0           None                 9.0         None 2018-02-26 21:36:10.121126   
1           None                 9.0         None 2018-02-28 19:59:09.996577   
2           None                 9.0         None 2018-03-01 19:15:01.789758   
3           None                 NaN         None 2018-02-20 20:02:02.656193   
4           None                 NaN         None 2018-02-20 20:01:24.158513   

  requested_date_range_start carousel_well_name      ...        \
0        2018-02-20 08:00:00               None      ...         
1        2018-02-20 08:00:00               None      ...         
2        2018-02-20 08:00:00               None      ...         
3        2018-02-20 08:00:00               None      ...         
4        2018-02-20 08:00:00               None      ...         

   tissue_processing_id  ephys_qc_result requested_by_id y_coord  \
0                  None             None       649368583    None   
1                  None             None       649368583    None   
2                  None             None       649368583    None   
3                  None             None       649368583    None   
4                  None             None       649368583    None   

  specimen_set_id cell_reporter_id   login  pinned_radius  \
0            None             None  nadiad           None   
1            None             None  nadiad           None   
2            None             None  nadiad           None   
3            None             None  nadiad           None   
4            None             None  nadiad           None   

  external_specimen_name parent_x_coord  
0                 360991           None  
1                 360906           None  
2                 360566           None  
3                 357433           None  
4                 356797           None  

[5 rows x 76 columns]

In [16]:
query = "SELECT scheduled_procedures.*, sps.subject_id, specimens.name AS mouse_name \
FROM scheduled_procedures \
JOIN users ON users.id = scheduled_procedures.assigned_to_id \
JOIN scheduled_procedures_subjects sps ON scheduled_procedures.id = sps.scheduled_procedure_id \
JOIN specimens ON sps.subject_id = specimens.id \
WHERE users.login = 'nadiad' and scheduled_procedures.lab_request_status ='active'"
df = get_lims_dataframe(query)
df.head()

task_type lab_request_status    task_id department_id  \
0      Task             active  665596389          None   
1      Task             active  665596637          None   
2      Task             active  665596826          None   
3      Task             active  665596853          None   
4      Task             active  665597226          None   

                  created_at                                mouse_name  \
0 2018-02-20 20:13:13.133917  Slc17a7-IRES2-Cre;Camk2a-tTA;Ai93-360991   
1 2018-02-20 20:13:17.837289                 Vip-IRES-Cre;Ai148-360906   
2 2018-02-20 20:13:22.027657       Cux2-CreERT2;Camk2a-tTA;Ai93-360566   
3 2018-02-20 20:13:27.512618                 Vip-IRES-Cre;Ai148-357433   
4 2018-02-20 20:13:38.828488  Slc17a7-IRES2-Cre;Camk2a-tTA;Ai93-356797   

                  updated_at requested_date_range_start  subject_id  \
0 2018-02-20 20:13:13.133917        2018-02-20 08:00:00   649497400   
1 2018-02-20 20:13:17.837289        2018-02-20 08:00:00   650072655   
2 2018-02-20 20:13:22.027657        2018-02-20 08:00:00   649502537   
3 2018-02-20 20:13:27.512618        2018-02-20 08:00:00   646205980   
4 2018-02-20 20:13:38.828488        2018-02-20 08:00:00   646498173   

                                          batch_name  assigned_to_id  \
0  dillanb_C600_Brain Observatory Visual Coding_2...             186   
1  dillanb_C600_Brain Observatory Visual Coding_2...             186   
2  dillanb_C600_Brain Observatory Visual Coding_2...             186   
3  dillanb_C600_Brain Observatory Visual Coding_2...             186   
4  dillanb_C600_Brain Observatory Visual Coding_2...             186   

   requested_by_id comments scheduled_date methods_versions_map  \
0        649368583     None           None                   {}   
1        649368583     None           None                   {}   
2        649368583     None           None                   {}   
3        649368583     None           None                   {}   
4        649368583     None           None                   {}   

   task_flow_step_id  project_id location_id         id  \
0          563649249   305493902        None  665596392   
1          563649249   305493902        None  665596646   
2          563649249   305493902        None  665596829   
3          563649249   305493902        None  665596856   
4          563649249   305493902        None  665597231   

  requested_date_range_end  
0      2018-02-20 08:00:00  
1      2018-02-20 08:00:00  
2      2018-02-20 08:00:00  
3      2018-02-20 08:00:00  
4      2018-02-20 08:00:00

In [17]:
#Jim asked to generate a csv with the rows being each patch-seq cell and the columns being the number 
#of sweeps for each of those cells that are of a certain stim.description (name)

query = "SELECT cell.name AS cell_name, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%subthreshold%%' THEN 1 ELSE NULL END) AS subthreshold \
FROM specimens cell JOIN ephys_sweeps ess ON cell.id = ess.specimen_id \
JOIN ephys_stimuli ON ess.ephys_stimulus_id = ephys_stimuli.id \
WHERE cell.patched_cell_container NOTNULL \
GROUP BY cell.name"

df2 = get_lims_dataframe(query)
df2.head()

subthreshold                              cell_name
0             3      Oxtr-T2A-Cre;Ai14-351471.04.01.01
1             0  Slc32a1-IRES-Cre;Ai14-326812.04.02.05
2             0  Slc32a1-IRES-Cre;Ai14-305535.06.02.01
3             0     Gad2-IRES-Cre;Ai14-267341.07.02.02
4             0  Slc17a6-IRES-Cre;Ai14-309388.03.01.01

In [18]:
query = "SELECT cell.name AS cell_name, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%subthreshold%%' THEN 1 ELSE NULL END) AS subthreshold, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1LSFINEST150112%%' THEN 1 ELSE NULL END) AS longsquare, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1SSFINEST150112%%' THEN 1 ELSE NULL END) AS shortsquare, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1RP25PR1S141203%%' THEN 1 ELSE NULL END) AS ramp, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%Blip%%' THEN 1 ELSE NULL END) AS cap_check, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C2CHIRP171103%%' THEN 1 ELSE NULL END) AS chirp, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1NSD1SHORT17110%%' THEN 1 ELSE NULL END) AS noise_1, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C1NSD2SHORT17110%%' THEN 1 ELSE NULL END) AS noise_2, \
COUNT(CASE WHEN ephys_stimuli.description LIKE '%%C2SSTRIPLE171103%%' THEN 1 ELSE NULL END) AS triple \
FROM specimens cell JOIN ephys_sweeps ess ON cell.id = ess.specimen_id \
JOIN ephys_stimuli ON ess.ephys_stimulus_id = ephys_stimuli.id \
WHERE cell.patched_cell_container NOTNULL \
GROUP BY cell.name"

df2 = get_lims_dataframe(query)
df2.head()

noise_1  ramp  shortsquare  chirp  noise_2  subthreshold  cap_check  \
0        0     2            5      0        0             3          1   
1        0     0            0      0        0             0          0   
2        0     1            3      0        0             0          0   
3        0     3            7      0        0             0          0   
4        0     1            3      0        0             0          0   

   longsquare  triple                              cell_name  
0           4       0      Oxtr-T2A-Cre;Ai14-351471.04.01.01  
1           4       0  Slc32a1-IRES-Cre;Ai14-326812.04.02.05  
2           1       0  Slc32a1-IRES-Cre;Ai14-305535.06.02.01  
3           0       0     Gad2-IRES-Cre;Ai14-267341.07.02.02  
4           0       0  Slc17a6-IRES-Cre;Ai14-309388.03.01.01

NameError: name 'CLOSE' is not defined